In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 39.04 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [ ]:
%tensorflow_version 1.x
!pip3 install h5py==2.10.0
import requests
import os
import sys
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import tensorflow
import cv2
from google.colab.patches import cv2_imshow

TensorFlow 1.x selected.


In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("/content/Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "/content/foodimages")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

Using TensorFlow backend.


In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [ ]:
# Load an image from the images folder
#file_names = next(os.walk(IMAGE_DIR))[2]
file_name = "/content/foodimages/food.jpg"
image = skimage.io.imread(file_name)

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]

Processing 1 images
image                    shape: (500, 500, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32



In [ ]:
api_user_token = 'feb16a6f08967f1fbad8952bb0293e1454f24ce6'
headers = {'Authorization': 'Bearer ' + api_user_token}
# Single Dishes Detection
url = 'https://api.logmeal.es/v2/recognition/dish'

limit=10
from PIL import Image
from os import path

for n in range(limit):
  if (path.exists(str(n) + ".jpg")):
    os.remove(str(n) + ".jpg")

img = cv2.imread(file_name)
#cv2_imshow(img)
mask = r['masks']
class_ids= r['class_ids']
class_names=None

# Pick top prominent classes in this image
unique_class_ids = np.unique(class_ids)
mask_area = [np.sum(mask[:, :, np.where(class_ids == i)[0]])
              for i in unique_class_ids]
top_ids = [v[0] for v in sorted(zip(unique_class_ids, mask_area),
                                key=lambda r: r[1], reverse=True) if v[1] > 0]
# Generate images and titles

for i in range(1, limit):
  if i<len(top_ids):
    class_id = top_ids[i]
    # Pull masks of instances belonging to the same class.
    m = mask[:, :, np.where(class_ids == class_id)[0]]
    m = np.sum(m * np.arange(1, m.shape[-1] + 1), -1)
    copy = np.zeros((len(img), len(img[0]), 3), dtype=np.uint8)
    for row in range(len(m)):
      for c in range(len(m[0])):
        if m[row][c]==1:
          copy[row][c]=[255, 255, 255]

    result = cv2.bitwise_and(img, copy)
    result[m==0] = 255
    for row in range(len(result)):
      for c in range(len(result[0])):
        if all(result[row][c] == [255, 255, 255]) == False:
          result[row][c] = img[row][c]
    cv2_imshow(result)
    cv2.imwrite('/content/' + str(i) + '.jpg', result)

    imgpath = '/content/' + str(i) + '.jpg'
    resp = requests.post(url,files={'image': open(imgpath, 'rb')}, headers=headers)
    try:
      data = (resp.json()["recognition_results"]) # display dish only
    except:
      print("Likely not a food")
    else:
      grayimg = cv2.imread(os.path.join(ROOT_DIR, file_name), cv2.IMREAD_GRAYSCALE)
      grayresult = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
      numOgNonWhitePix = len(grayimg) * len(grayimg[0]) - np.sum(grayimg == 255)
      numResultNonWhitePix = len(grayimg) * len(grayimg[0]) - np.sum(grayresult == 255)
      fraction = numResultNonWhitePix/numOgNonWhitePix
      foods = []
      for i in range(len(data)):
        foods.append([data[i]['name'], fraction])
      print(foods)


[['grilled chicken', 0.2228455874727782], ['icetea', 0.2228455874727782], ['grilled turkey', 0.2228455874727782], ['pineapple juice', 0.2228455874727782], ['orange soda', 0.2228455874727782], ['stuffed eggplant', 0.2228455874727782]]
Likely not a food


python3 version: 3.7

> click and run the file called command certificates or smth

tensorflow version: 1.13.2

> download *tensorflow-1.11.0-cp27-cp27m-macosx_10_11_x86_64.whl*

h5py version: 2.10.0

In [ ]:
'''
    resp = requests.post(url,files={'image': open(imgpath, 'rb')}, headers=headers)
    try:
      data = (resp.json()["recognition_results"]) # display dish only
    except:
      print("An exception occurred, likely not a food")
    else:
      foods = []
      for i in range(len(data)):
        foods.append(data[i]['name'])
      print(foods)
      edited = cv2.cvtColor(result, cv2.COLOR_BGR2HSV)
      thresh = cv2.inRange(edited, (0,0,0), (254,254,254));
      contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      contours = contours[0] if len(contours) == 2 else contours[1]
      cntr = max(contours, key=cv2.contourArea)
      cntr = cntr[1::3]
      for c in range(len(cntr)):
        cntr[c] = cntr[c].flatten()
      if (i==0):
        info["shapes"].append({
            'label': "plate",
            'points': cntr.tolist()
        })
      else:
        info["shapes"].append({
            'label': i,
            'points': cntr.tolist()
        })

with open('data.json', 'w') as outfile:
    json.dump(info, outfile)
'''